# MPCS 53111 - homework 4

## Name: Zhenyang Lu

# question 2 

Notice: The way I did this homework is not designing it first then implementing it. It's rather the opposite like reverse engineering. I honestly don't know how to design it from theory. So below is best I can do. 

## Design: 

Activation function: Linear 

Three input variables: 

$X_1$        

$X_2$

$X_1*X_2$

Hidden Layer: 

Neuron 1: take $X_1$ with weights -1, $X_1*X_2$ with weights 1, bias = 0.19

Neuron 2: take $X_2$ with weights -1, $X_1*X_2$ with weights 1, bias = 0.1

Final Output: 

take 0.41 weight from Neuron 1, 0.52 weight from Neuron 2. 

# Question 3

when considering possibility of incorrect setting. The two parts of the original formular needs to be modified to include possibility of error :

$(y^i*log(h(x^i))$ should be becomes $y^i*(log(h(x^i)(1-\delta) + (1-h(x^i))*\delta)) $

$(1-y^i)log(1-h(x^i)))$ should be like $(1-y^i)*log((1-h(x^i))(1-\delta)+h(x^i)*\delta)$

So combining together the two scenarios, we have  

$l(\theta) = \sum^m_{i=1} = y^i*log(h(x^i)(1-\delta) + (1-h(x^i))*\delta) + (1-y^i)*log((1-h(x^i))(1-\delta)+h(x^i)*\delta)$

which should be the same as the original formula that Andrew Ng derives if $\delta = 0$

# Question 4 

I'm very confused by this question. It seems like very straightforward to me, what the neuron network tries to predict (also, why the author calls it a back propagation network, rather than Neuro network, or there is some difference between the two). It's just 80% of time it predicts 1, and 20% it predicts 0, so if it's equally weighted and independent with each other, its expected prediction is 0.8.

But following the hint given, which is totally another direction of thinking, it's also very straightforward: 

Define X to be the true label for the only example. Cost function: 

$J(\theta) = \frac{1}{2} [80*(1-X)^2 + 20(0-X)^2] $, if taken derivative with respect to X (I don't think we should take derivative with respect to the predicted value h(x) because it's either 0 or 1. The only independent variable is X.), we get 100X-80, so X is still 0.8. 

In [1]:
# Question 5

import pandas as pd 
import numpy as np
import matplotlib as plt

raw = pd.read_csv("C:\\Users\\zheny\\Downloads\\train.csv")
raw.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Question 5

For this question, there are three objects: Neuron, Layer(contains Neurons) and ANN( artificial network, which contains Layers). Following is Neuron, variable definitions and method functioning is given.

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

class Neuron:
    # size of weights = num_nodes_last_layer*num_weight
    # b is uniform dist parameter Xavier initialization,  
    # b = 6/2*(number of nodes in each layer)**(1/2)
    
    def __init__(self, num_weight, b,bias=1):
        
        self.bias = bias
        # input is a list such that each item is a pd.DataFrame
        self.input = None
        # contains g(W*a)
        self.output = None
        # input dataframe size
        self.input_shape = None
        self.output_shape = None
        # weights vector
        self.weights = None 
        # weights is a list such that each item is a pd.DataFrame
        # same size as self.inputs
        
        # size of weights is the same as size of a picture(1,M)
        self.num_weight = num_weight
        # this is g'(W*a)       
        self.delta = None 
        
        if self.weights is None:
            self.weights = pd.DataFrame(np.random.uniform(-b,b,self.num_weight))
        
    # calculate neuron's delta when back propagation for outout layer
    def output_neuron_delta(self,error):
        # delta is a number type
        self.delta = error*self.derivative()

    # calculate neuron's delta when back propagation for hidden layer
    def hidden_layer_delta(self,weight,next_delta):
        self.delta = self.derivative()*weight*next_delta

    # calculate Neuron's output, g(W*a+bias)
    def calculate_out(self, inputs):
        self.input = inputs
        self.input_shape = inputs.shape
        self.output = self.logistic_regression(self.calculate_in())
        self.output_shape = (1,1)
        
        return self.output

    # calculate W*a + bias
    def calculate_in(self):                
        # inputs[1] and weights[1] are DataFrame
        return np.dot(self.input, self.weights)[0][0] + self.bias

    # calculate g'(a)
    def derivative(self,epsilon = 0.001):
    
        return (self.logistic_regression(self.calculate_in()+epsilon) - \
                self.logistic_regression(self.calculate_in()-epsilon))/(2*epsilon)

    # logistic function, well it shouldn't be logistic_regression, rather it's logistic
    def logistic_regression(self, input):
        return 1 / (1 + np.exp(-input))



This is Layer objects

In [3]:
# each Layer object contains neuron_num numbers of neuron objects
class Layer:
    # neuron_num is the number of neurons within this layer
    # weight_num is how many weights for each neuron
    def __init__(self, neuron_num, weight_num, bias=1):

        # default bias = 1
        self.bias = bias 
        # list of neurons in this layer
        self.neurons = []
        # how many neurons 
        self.neuron_num= neuron_num
        # self.output is the output that feeds to the next layer
        self.output = None
        # self.weight_num is how many weights for each neuron in this layer
        self.weight_num = weight_num
    
        # random initialize weights Xavier Initialization
        for i in range(neuron_num):
            self.neurons.append(Neuron(self.weight_num,np.sqrt(6/(self.neuron_num+self.neuron_num)),self.bias))

    # calculate the output for each neuron
    # store it in self.output
    def feed_forward(self, inputs):
        self.output = pd.DataFrame([])
        
        for neuron in self.neurons:
            self.output = self.output.append([neuron.calculate_out(inputs)],ignore_index=True)
        
    # return output to next layer
    def feed_output(self):
        return self.output   

## please read this page! 

## Assumptions and Discussion about Question 5 

this is ANN, artificial neuron network. There is much to talk about this object. 

This ANN is not really like how the homework wants me to do. Specifically, A single ANN object is not able to take in a bunch of images and tell which image is what digit, rather 10 ANN objects are needed and each ANN object can only be used to tell how likely for an image to be a specific digit. 

I took a look at many ANN example, but I was confused how can I use a network with more than one output neuron in the output layer. But, my way of implementation can actually form a large network which can guess what an image would be if there are 10 ANN objects get trained, each of which tells the likeliness for an image to be 1,2,3.. etc, only with the cost of excessive computation. 

I'm sure there is a way to train a large network that can tell the handwritting for each picture, but I don't quite understand how it works. So for question 6, I will use a network like this: 


0. There are 10 ANN objects, each is responsible for telling how likely an input image would be a digit. So for example, when an image will be put into 10 ANN objects, the first ANN object tell how likely is it to be "0", second tells how likely is it to be a "1"....etc.  
1. there are 3 layers in each ANN:input, output and one hidden layer. 
2. input layer: take as input a bunch of images, each of which is a row of record in MNIST. 
3. hidden layer: number of nodes are specified when calling the constructor.
4. output layer: contains just one neuron, whose output is a likeliness tells percentage of likeliness the input picture is one of (1,2,3,4,5,6,7,8,9).  So if I want to test a image dataset datasets with 9 labels, then I will need to train 9 ANN objects and put the image to each of them. This is maybe taking more time to train and test, and sounds like a silly way to do it. 

So if you want to test out ANN, remember that an ANN object can only tell how likely an image is one of [1,2,3,4,5,6,7,8,9,0], depending only which one of [1,2,3,4,5,6,7,8,9,0] you train it to tell.

I will provide a auxiliary function to help test MNIST in question 6. For now please don't expect it to be exactly like the homework's specification. 

Below is ANN objects: 

In [4]:
class ANN():
    
    # h number of hidden layer
    # s number of nodes in each layer
    def __init__(self,s,h=1):
        
        # y_hat: predicted value 
        self.y_hat = None
        # y: input 
        self.y = None
        # number of HIDDEN+output layers, besides input layer
        self.h = h
        # number of nodes in each layer
        self.s = s
        # learning rate
        self.alpha = None 
        #input 
        self.X = None
        # number of iterations
        self.t = None
        # number of input images
        self.num_input = None
        # number of output labels
        self.num_output = None
        # num_column is how many columns in self.xX
        self.num_column = None
        # a list of hidden layers
        self.hidden_layers = []
        # output neuron 
        self.output_neuron = Neuron(num_weight = self.s,b = np.sqrt(6/(1+self.s)))
        
            
    # print(h=1,s=1) prints out weights of 1st node in hidden layer 1
    # print(h=1) prints out all nodes's weights in layer 1
    # print(s=1) prints out 1st node in all layers
    def print(self,h=None,s=None):

        # print weight of all nodes in the current ANN
        if h is None and s is None:
            for i in range(len(self.hidden_layers)):
                for j in range(len(self.hidden_layers[i].neurons)):
                    print("Weights in",j+1, "\bth neuron in",i+1,"\bth hidden layer: \n",\
                          self.get_node_weight(i,j).T.values,"\n")
        
        # print weights of nodes in a specified layer
        if h is not None and s is None:
            for j in range(len(self.hidden_layers[h].neurons)):
                print("Weights in",j+1, "\bth neuron in",h+1,"\bth hidden layer: \n",\
                          self.get_node_weight(h,j).T.values,"\n")
    
        if h is not None and s is not None:
            print("Weights in",s+1, "\bth neuron in",h+1,"\bth hidden layer: \n",\
                          self.get_node_weight(h,s).T.values,"\n") 
        
    
    # get weights of the s-th node in the h-th hiddent layer
    # both starting from 1! 
    def get_node_weight(self,h,s):
        
        if h < 0 or s < 0:
            print("h and s can not be smaller than 1")
            return 
        
        if h > self.h -1 :
            print("Invalid number of hidden layer, max level of h(layers): ",self.h-1)
            return 
        
        if s > self.s-1:
            print("Invalid number of node, max level of s(nodes): ",self.s-1)
            return 
        
        return self.hidden_layers[h].neurons[s].weights

    # return weights on the output neuro
    def get_output_neuro_weight(self):
        return self.output_neuron.weights

    # fit a model. 
    def fit(self,X,y,alpha,t):
        self.X = X
        self.y = y
        self.alpha =alpha
        self.t = t
        # number of input images
        self.num_input = self.X.shape[0]
        # number of output labels
        self.num_output = self.X.shape[0]
        self.num_column = self.X.shape[1]

        # initiating hidden layer and weights of its neurons
        for i in range(self.h):
            self.hidden_layers.append(Layer(self.s,self.num_column))

        self.back_propagation()

    # take an image and output a guess. 
    def predict(self,T):
        
        y_hat = pd.DataFrame([])
        
        # loop through each example
        for j in range(self.X.shape[0]):
            
            # aviod pass-by-reference
            next_input = self.X.iloc[j:j+1,:].copy()
            
            # for each example, do the forward propagation
            for i in self.hidden_layers:
                i.feed_forward(next_input)
                next_input = i.feed_output().copy()
            
            self.output_neuron.calculate_out(next_input.T)
                
            y_hat = y_hat.append(pd.DataFrame([self.output_neuron.output]),ignore_index=True)
        return y_hat
                
        
    # back propagation     
    def back_propagation(self):
        
        # number of iterations 
        for i in range(self.t):
            
            # loop through each example
            for j in range(self.X.shape[0]):
                
                # aviod pass-by-reference
                next_input = self.X.iloc[j:j+1,:].copy()
                
                # for each example, do the forward propagation
                for i in self.hidden_layers:
                    i.feed_forward(next_input)
                    next_input = i.feed_output().copy()
                
                self.output_neuron.calculate_out(next_input.T)
                  
                # back propagation output Neuron
                self.y_hat = pd.DataFrame([self.output_neuron.output])
                # error is a number 
                self.error = (self.y.iloc[j,:][0] - self.y_hat)[0][0]

                #self.before_weights = self.output_neuron.weights.copy()
                self.output_neuron.output_neuron_delta(self.error)             
                
                #print(0.5*(self.error)**2)
                
                # Since Question 6 asks to avoid cases with more than 1 
                # hidden layer, I will here try to be lazy here, this only
                # works for case with only 1 hidden layer 
                for i in range(len(self.hidden_layers[0].neurons)):                 
                    
                    # update hidden layer's weights
                    self.hidden_layers[0].neurons[i].hidden_layer_delta(\
                    self.output_neuron.weights.iloc[i,:][0],self.output_neuron.delta)
                    
                    change = (self.alpha*self.hidden_layers[0].neurons[i].input*\
                    self.hidden_layers[0].neurons[i].delta).T
                    
                    change.reset_index(drop=True,inplace=True)
                    change.columns=[0]
                      
                    self.hidden_layers[0].neurons[i].weights =  \
                    self.hidden_layers[0].neurons[i].weights + change
                
                    self.before = self.output_neuron.weights.copy()
                
                    # update the output neuron's weights
                    self.output_neuron.weights = self.output_neuron.weights + \
                         (self.alpha*self.output_neuron.input*\
                         self.output_neuron.delta).T
                          
                    #print(self.output_neuron.weights)

# Question 6

## Notice: you must use exactly train.csv and test.csv from the kaggle project link given by the homework description. Using the original MNIST dataset will not be ok.  
Given the weird design that mentioned in Question (sorry again for it). I will need an auxiliary functionallities(which is below) to put things together so that when a dataset of images is given, a similar output required by predict() function in Question 5 will be output. 

some empirical testing experience: The speed depends on the number of training images, size of hidden layer and most heavily on number of iterations. On my own testing, an ANN object takes about 2 minutes to train a dataset with 500 images and 10 iterations, with learning rate = 0.05 using a single hidden layer with 10 nodes. But this learning rate is inferior than a rate of 0.01, combined with 50 iterations in terms of accuracy rate. So I would take a training size 500 images for each ANN model with iteration about 10 - 50 times. iteration 0.01 - 0.05. Each ANN model takes about 2 minutes to train, and there are 10 ANN models needed (each is responsible for telling the likeliness of resembling a single digits).

So the following test parameters works better in terms of accuracy for each ANN object: 

       iteration: 20 - 30   
       training_size: 500 or more
       learning: 0.01
       nodes in hidden layer: 10 or more
       
Totally, there 10 ANN objects needed, on my own test, the above case will need 20 minutes to train. Please consider how much time you would spend on grading each homework and adjust parameters. 

In [5]:
# Let's train 10 ANN objects, each of which is trained to tell the likeliness an image is one of [0,1,2,3,4,5,6,7,8,9].

# input is a dataframe input

# original MNIST data in CSV files.

# models is a list to contain 10 trained ANN objects.
# for now assume that we are using just 1 hidden layer
import time

def train_model(raw_file, models,iteration,training_size,learning_rate,node_in_hidden_layer):
    
    raw = pd.read_csv(raw_file)  
    # this list contains 10 trained ANN objects that can be used to predict

    for i in range(10):
        
        print("Now training model for digit: ", i)
        before= time.time()
        
        # random choose sample images from original dataset
        # to be used as training images for model 
        a = raw[raw['label']==i].sample(training_size*2) 
        a.reset_index(drop=True,inplace=True)

        na = raw[raw['label']!=i].sample(training_size*2) 
        na.reset_index(drop=True,inplace=True)
        na.iloc[:,0:1] =0

        c = pd.concat([a,na])
        c.reset_index(drop=True,inplace=True)

        random_c = c.sample(training_size)
        random_c.reset_index(drop=True,inplace=True)

        # end of random sampling
        
        # X is input X 
        # y is actual labels of X 
        X = random_c.iloc[:,1:]
        y = random_c.iloc[:,0:1]
        
        # train model   
        models.append(ANN(s=node_in_hidden_layer))
        models[i].fit(X,y,learning_rate,iteration)
        
        print("Finished training model for digit: ", i)
        print("Time used:", time.time()-before)
        
### train models ###

model = []

train_model(raw_file ="C:\\Users\\zheny\\Downloads\\train.csv", \
            models = model,iteration=10,training_size=500,learning_rate=0.01,node_in_hidden_layer=10)

Now training model for digit:  0
Finished training model for digit:  0
Time used: 847.6264219284058
Now training model for digit:  1
Finished training model for digit:  1
Time used: 839.7619171142578
Now training model for digit:  2
Finished training model for digit:  2
Time used: 847.0323033332825
Now training model for digit:  3
Finished training model for digit:  3
Time used: 842.4923000335693
Now training model for digit:  4
Finished training model for digit:  4
Time used: 942.9699575901031
Now training model for digit:  5
Finished training model for digit:  5
Time used: 965.4952714443207
Now training model for digit:  6
Finished training model for digit:  6
Time used: 1018.8610332012177
Now training model for digit:  7
Finished training model for digit:  7
Time used: 932.4615404605865
Now training model for digit:  8
Finished training model for digit:  8
Time used: 845.6873750686646
Now training model for digit:  9
Finished training model for digit:  9
Time used: 989.1709711551666

In [13]:
def predict(X,y,model):
    result = pd.DataFrame([])
    
    for i in range(X.shape[0]):
        row_result = []
        
        for each_model in model:
            row_result.append(each_model.predict(X.iloc[i:i+1,:])[0][0])            

        row_result.append(y.iloc[i,:][0])
        row_result = pd.DataFrame([row_result])
        result = pd.concat([result,row_result],ignore_index=True)
        
    return result

# prepare for test data 
# random choose 100 images from MNIST and try to predict it

raw = pd.read_csv("C:\\Users\\zheny\\Downloads\\train.csv")
test = raw.sample(100) 
test.reset_index(drop=True,inplace=True)
X = test.iloc[:,1:]
y= test.iloc[:,0:1]

# X is input 
# y is label
# model is a list contains the models trained 
predict(X,y,model) # predict should relatively quick 

KeyboardInterrupt: 